In [14]:
# coding: utf-8
"""
attention modelで文書分類
# 未完成
"""
import sys
import numpy
from argparse import ArgumentParser
from chainer import Chain, Variable, cuda, functions, links, optimizer, optimizers, serializers
import collections
import numpy as np
import random
#import util.generators as gens
#from util.functions import trace, fill_batch
#from util.vocabulary import Vocabulary

In [119]:
#Added comment

def make_corpus(list_corpus, lower_freq=1):
    list_all_word = [word for row in list_corpus for word in row]
    list_word_freq = collections.Counter(list_all_word)
    # 頻度がlower_freq以下の単語を<unk>に変える
    list_unk_word = [word for word, freq in list_word_freq.items() if freq <= lower_freq]
    list_corpus_rev = [['<s>']+['<unk>' if word in list_unk_word else word for word in row]+['</s>'] for row in list_corpus]
    # 辞書を作成
    dict_word_id = {'<s>': 0, '</s>':1, '<unk>':2}
    for row in list_corpus_rev:
        counter = 3
        for word in row:
            if word not in dict_word_id:
                dict_word_id[word] = counter
                counter += 1
    # 作成した辞書を用いて、corpus中の単語をidに変更
    list_corpus_id = [[dict_word_id[word] for word in row] for row in list_corpus_rev]
    # 作成したコーパスと辞書を返す
    return list_corpus_id, dict_word_id

class SrcEmbed(Chain):
    def __init__(self, vocab_size, embed_size):
        super(SrcEmbed, self).__init__(
            xe = links.EmbedID(vocab_size, embed_size),
        )

    def __call__(self, x):
        return functions.tanh(self.xe(x))


class LSTMEncoder(Chain):
    def __init__(self, embed_size, hidden_size):
        super(LSTMEncoder, self).__init__(
            lstm = links.LSTM(embed_size, hidden_size),
        )
    def reset(self):
        self.zerograds()
    def __call__(self, x):
        h = self.lstm(x)
        return h

class Attention(Chain):
    def __init__(self, hidden_size):
        super(Attention, self).__init__(
            pw = links.Linear(hidden_size, hidden_size),
            we = links.Linear(hidden_size, 1),
        )
        self.hidden_size = hidden_size
    
    def __call__(self, a_list):
        e_list = []
        self.empha = []
        sum_e = Variable(np.array([[0]], dtype='float32'))
        for a in a_list:
            w = functions.tanh(self.pw(a))
            e = functions.exp(self.we(w))
            e_list.append(e)
            sum_e += e

        ZEROS = Variable(np.zeros((1, self.hidden_size), dtype='float32'))
        aa = ZEROS
        for a, e in zip(a_list, e_list):
            e /= sum_e
            self.empha.append(e)
            aa += a * functions.broadcast_to(e, (1, self.hidden_size))
        #aa += functions.reshape(functions.batch_matmul(a, e), (batch_size, self.hidden_size))
        return aa

class AttentionLM(Chain):
    def __init__(self, embed_size, hidden_size, vocab_size, label_size):
        super(AttentionLM, self).__init__(
            emb = SrcEmbed(vocab_size, embed_size),
            enc = LSTMEncoder(embed_size, hidden_size),
            att = Attention(hidden_size),
            outae = links.Linear(hidden_size, hidden_size),
            outey = links.Linear(hidden_size, label_size),
        )
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.label_size = label_size

    def reset(self):
        self.zerograds()
        self.x_list = []
        self.h_list = []

    def embed(self, x):
        self.x_list.append(self.emb(x))

    def encode(self):
        for x in self.x_list:
            self.h = self.enc(x)
            self.h_list.append(self.h)

    def decode(self):
        aa = self.att(self.h_list)
        y = functions.tanh(self.outae(aa))
        return self.outey(y)

In [129]:
def get_data(variable):
    #return variable.data
    return cuda.to_cpu(variable.data)

def forward(list_corpus_one, label, model):
    L = len(list_corpus_one)

    opt.zero_grads()

    for one in list_corpus_one:
        model.embed(Variable(np.array([one], dtype='int32')))
    s_t = Variable(np.array([label], dtype='int32'))
    model.encode()
    s_y = model.decode()
    loss_i = functions.softmax_cross_entropy(s_y, s_t)
    
    return loss_i
            

def main():
    list_corpus_id, dict_word_id = make_corpus(list_corpus, lower_freq=1)
    list_corpus_id_y = [[row, y] for row, y in zip(list_corpus_id, list_y)]

    model = AttentionLM(embed_size=300,
                        hidden_size=100,
                        vocab_size=len(dict_word_id),
                        label_size=3)
    model.reset()

    for epoch in range(epoch):
        log_ppl = 0.0
        trained = 0
        
        opt = optimizers.AdaGrad(lr = 0.01)
        opt.setup(model)
        opt.add_hook(optimizer.GradientClipping(5))
        # コーパスのシャッフル
        random.shuffle(list_corpus_id_y)

        for list_corpus_one, y in list_corpus_id_y:
            loss, perplexity= forward(list_corpus_one=list_corpus_one,
                                      label=y,
                                      model=model)
            loss.backward()
            log_ppl += perplexity 
            opt.update()
            model.reset()

        serializers.save_hdf5('%s.weights'%str(epoch), model)

#if __name__ == '__main__':
#    main()

In [128]:
model = AttentionLM(embed_size=10,
                    hidden_size=5,
                    vocab_size=20,
                    label_size=3)
model.reset()

opt = optimizers.AdaGrad(lr = 0.01)
opt.setup(model)
opt.add_hook(optimizer.GradientClipping(5))

for num in range(5):
    model.embed(Variable(np.array([num], dtype='int32')))
    
model.encode()
s_y = model.decode()

s_t = Variable(np.array([0], dtype='int32'))
loss_i = functions.softmax_cross_entropy(s_y, s_t)
loss_i.backward()
opt.update()
model.reset()

In [132]:
model = AttentionLM(embed_size=10,
                    hidden_size=5,
                    vocab_size=20,
                    label_size=3)
model.reset()

opt = optimizers.Adam()
opt.setup(model)
opt.add_hook(optimizer.GradientClipping(5))

list_corpus_one = [1,2,3,4,5]

loss = forward(list_corpus_one=list_corpus_one,
                          label=1,
                          model=model)
loss.backward()
opt.update()
model.reset()

In [118]:
#print model.emb.xe.W.data
print model.emb.xe.W.grad
#print model.enc.lstm.c
#print model.enc.lstm.h
#print model.att.pw.W.data
print model.att.pw.W.grad
#print model.att.we.W.data
print model.att.we.W.grad
#print model.outae.W.data
print model.outae.W.grad
#print model.outey.W.data
print model.outey.W.grad

[[ -6.99495291e-03   4.52087075e-03   2.84918607e-03   2.08294904e-03
   -2.49932483e-02  -9.31231957e-03  -2.92772576e-02   6.61760196e-03
   -1.41288862e-02   1.65105797e-02]
 [  1.09188573e-03   1.17711555e-02  -1.21388007e-02  -1.05180847e-03
   -1.15289073e-02  -1.61390249e-02  -1.75921731e-02  -1.12938248e-02
   -2.30505075e-02   6.98590046e-03]
 [ -5.69005078e-03   1.23687144e-02   5.15864231e-03  -2.93265330e-03
   -8.80991574e-04  -5.00305323e-04  -4.55614813e-02   2.33042948e-02
   -1.96060650e-02   1.35435408e-03]
 [  4.80962153e-05  -5.54220984e-03   2.34324648e-03  -6.74829178e-04
   -5.72424615e-03  -4.53865947e-03  -7.88539182e-03   3.05371289e-03
   -6.51086541e-03   6.73896214e-03]
 [  2.44483957e-03  -7.21206982e-03   1.07384613e-03  -4.65594186e-03
   -4.13720272e-02  -6.19655987e-03  -2.09817983e-04  -1.31178563e-02
   -1.62142571e-02  -7.11290864e-04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000

In [92]:
x = Variable(np.array([[1,2,3]], dtype='float32'))
y = Variable(np.array([[5]], dtype='float32'))
y_rev = functions.broadcast_to(y, (1,3))
print x.data
print y.data
print y_rev.data
z = x * y
print z.data

[[ 1.  2.  3.]]
[[ 5.]]
[[ 5.  5.  5.]]


InvalidType: 
Invalid operation is performed in: _ * _ (Forward)

Expect: in_types[0].shape == in_types[1].shape
Actual: (1, 3) != (1, 1)